In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-10-30 17:15:01--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2023-10-30 17:15:01 (7.86 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-10-30 17:15:01--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train=pd.read_csv(train_file_path,sep='\t',names=['Spam','message'])
test=pd.read_csv(test_file_path,sep='\t',names=['Spam','message'])
df=pd.concat([train,test])
df.reset_index(drop=True,inplace=True)

In [ ]:
df.head()

,Spam,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Spam     5571 non-null   object
 1   message  5571 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
words=df.message[0]
print(words)

ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.


In [ ]:
def preprocessing(message):
    words=re.sub(r'[^a-zA-Z\s]','',message)
    words=word_tokenize(words)

    stop_words=set(stopwords.words('english'))

    words=[W for W in words if W not in stop_words]

    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in words]

    return' '.join(words)

In [ ]:
df['message']=df['message'].apply(preprocessing)
df['Spam'].replace(['ham','spam'],[0,1],inplace=True)
df.head()

,Spam,message
0,0,ahhhhjust woken uphad bad dream u thoso dont l...
1,0,never noth
2,0,u sound like manki scous boy stevelik travel d...
3,0,mum say wan go go shun bian watch da glass exh...
4,0,never lei v lazi got wat dat day send da url c...


In [ ]:
#max_words=max(len(message) for message in df['message'])

max_words=df['message'].apply(len)
print(max_words.max())
print(max_words.idxmax())

477
1139


In [ ]:
X=df['message']
y=df['Spam']


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=28)

In [ ]:
#Creating Tokenizer
tokenizer =keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

#Converting text to sequence integers
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure they have the same length
max_sequence_length=100
X_train=keras.preprocessing.sequence.pad_sequences(X_train,maxlen=max_sequence_length, padding='post', truncating='post')
X_test=keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')


In [ ]:
#creating nureal network
#Embedding layer used to learn and represent words or categorical data in a continuous, lower-dimensional space especially used in NLP.
model=keras.Sequential([
    keras.layers.Embedding(input_dim=5000,output_dim=16),
    keras.layers.Flatten(),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


#Binary Cross-Entropy = - [y * log(p) + (1 - y) * log(1 - p)] >>y true label(0,1) , p is predicted probability
#This loss function is used for binary classification especially with sigmoid

In [ ]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8699 - loss: 0.3796 - val_accuracy: 0.9345 - val_loss: 0.1683
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9584 - loss: 0.1662 - val_accuracy: 0.9785 - val_loss: 0.1310
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9843 - loss: 0.1219 - val_accuracy: 0.9812 - val_loss: 0.1228
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9930 - loss: 0.0963 - val_accuracy: 0.9821 - val_loss: 0.0986
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9967 - loss: 0.0311 - val_accuracy: 0.9848 - val_loss: 0.0580


In [ ]:
new_sentence = "Congratulations! You've won a free vacation. Click the link to claim your prize."
new_sentence_sequence = tokenizer.texts_to_sequences([new_sentence])
new_sentence_padded = keras.preprocessing.sequence.pad_sequences(new_sentence_sequence, maxlen=max_sequence_length, padding='post', truncating='post')
prediction = model.predict(new_sentence_padded)

if prediction >= 0.5:
    print("The sentence is predicted as SPAM.")
else:
    print("The sentence is predicted as NOT SPAM (HAM).")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
The sentence is predicted as SPAM.


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9801 - loss: 0.0686
Test Accuracy: 0.9847533702850342


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text=preprocessing(pred_text)
  pred_text=tokenizer.texts_to_sequences([pred_text])
  pred_text=keras.preprocessing.sequence.pad_sequences(pred_text,maxlen=max_sequence_length, padding='post', truncating='post')

  pred=model.predict(pred_text)

  if pred>=0.5:
     prediction=[pred,'spam']
  else:
     prediction=[pred,'ham']

  return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[array([[0.99031514]], dtype=float32), 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]
  results=[]
  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    results.append(prediction[1])
    if prediction[1] != ans:
      passed = False
  print(results)
  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
['ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'ham']
You passed the challenge. Great job!
